# Main Notebook for Obesity Data Analysis Project

## Import Necessary Libraries

In [ ]:
import requests
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Set default visualization style
sns.set_style('whitegrid')
sns.set_palette('deep')



## Load Data

In [ ]:
df = pd.read_csv('../data/who_obesity_core_data.csv')
df.shape